In [125]:
import numpy as np 
import copy
import re
import sys
import json

x = {}
with open("hmmmodel.txt", "r") as file:
    file_content = file.read()
    x = json.loads(file_content)

emision_matrix = x["e"]
transition_matrix = x["t"]
blank = set(transition_matrix.keys())

In [126]:
def tag_a_line(line):
    output_matrix = []
    blank = {k: [0,""] for k in transition_matrix.keys()}
    prev_word = ""

    for i in range(len(line)):
        output_matrix.append(copy.deepcopy(blank))
        if i == 0:
            if(line[i] not in emision_matrix.keys()):
                line[i] = "__________"
            
            for pos in output_matrix[i].keys():
                tag = ''
                result = 0
                for pos_prev in blank:
                    prev_result = result
                    t = transition_matrix[pos_prev][pos]
                    e = emision_matrix[line[i]][pos]
                    result = max(result, t * e)
                    tag =  pos_prev if (result != prev_result) else tag

                output_matrix[i][pos] = [result,tag]
            
            flag_first = 0
            prev_word = i
            continue

        tag = ''
        result = 0
        for pos in output_matrix[i].keys():
            tag = ''
            result = 0
            for pos_prev in output_matrix[prev_word].keys():
                prev_result = result
                prior = output_matrix[prev_word][pos_prev][0]
                #prior = output_matrix[prev_word][pos_prev][0] if(output_matrix[prev_word][pos_prev][0] != 0) else alpha
                t = transition_matrix[pos_prev][pos]
                if(line[i] not in emision_matrix.keys()):
                    line[i] = "__________"
                e = emision_matrix[line[i]][pos]
                result = max(result, prior * t * e)
                tag =  pos_prev if (result != prev_result) else tag

            output_matrix[i][pos] = [result,tag]
            
        prev_word = i

    #output_matrix.keys()
    solution = []
    x = max(output_matrix[-1], key=output_matrix[-1].get)
    #print(x)
    solution.append(x)
    for i in range(len(line)-2,-1,-1):
        #print("Pos linked is: ", output_matrix[i+1][solution[-1]][1])
        #print("Pos Suggested is: ", output_matrix[i][solution[-1]][1])
        solution.append(output_matrix[i+1][solution[-1]][1])
    return solution

In [127]:
x = "\" Lei ha la tipica camminata da spia \" ."
y = tag_a_line(x.split())
y.reverse()
y

['FB', 'PE', 'V', 'RD', 'A', 'S', 'E', 'S', 'FB', 'FS']

In [128]:
#fp = open("hmm-training-data/it_isdt_dev_raw.txt","r", encoding='utf8')
filename = sys.argv[1]
fp = open(filename,"r", encoding='utf8')
lines = fp.readlines()
for line in lines:
    x = line.strip().split()
    s = tag_a_line(x.copy())
    s.reverse()
    with open('hmmoutput.txt', 'a', encoding='utf8') as f:
        for i in range(0,len(x)):
            f.write(x[i])
            f.write("/")
            f.write(s[i])
            f.write(" ")
        f.write("\n")
f.close()

In [80]:
"/FB Lei/FB ha/FB la/FB tipica/FB camminata/FB da/FB spia/FB "/FB ./FS 
"/FB Lei/PE ha/V la/RD tipica/A camminata/S da/E spia/S "/FB ./FS
['FB', 'PE', 'V', 'RD', 'A', 'S', 'E', 'S', 'FB', 'FS']

SyntaxError: invalid syntax (<ipython-input-80-50efa168918d>, line 1)